In [ ]:
import os
import sys
import numpy as np
import keras as K
import pickle
import tarfile
from urllib.request import urlretrieve
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from sklearn.preprocessing import OneHotEncoder

### Set env

In [ ]:
# TensorFlow,Theano,CNTK
os.environ['KERAS_BACKEND'] = "tensorflow" #Use TF1,some incompatibilities with TF2.
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Performance Improvement
# Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

### Load dataset

In [ ]:
def read_batch(src):
    '''Unpack the pickle files'''
    with open(src, 'rb') as f:
        if sys.version_info.major == 2:
            data = pickle.load(f)
        else:
            data = pickle.load(f, encoding='latin1') # Contains the numpy array
    return data

def process_cifar():
    '''Read data into RAM'''
    print('Preparing train set...')
    train_list = [read_batch('./cifar-10-batches-py/data_batch_{0}'.format(i + 1)) for i in range(5)]
    x_train = np.concatenate([x['data'] for x in train_list])
    y_train = np.concatenate([y['labels'] for y in train_list])
    print('Preparing test set...')
    tst = read_batch('./cifar-10-batches-py/test_batch')
    x_test = tst['data']
    y_test = np.asarray(tst['labels'])
    return x_train, x_test, y_train, y_test

def maybe_download_cifar(src="http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"):
    '''Load the training and testing data'''
    try:
        return process_cifar()
    except:
        # Catch the exception that file doesn't exist & Download
        print('Data does not exist. Downloading ' + src)
        filename = src.split('/')[-1]
        filepath = os.path.join("./",filename)
        def _recall_func(num,block_size,total_size):
            sys.stdout.write('\r>> downloading %s %.1f%%' % (filename,float(num*block_size)/float(total_size)*100.0))
            sys.stdout.flush()
        fname, h = urlretrieve(src, filepath,_recall_func)
        file_info = os.stat(filepath)
        print('Successfully download.',filename,file_info.st_size,'bytes')
        print('Extracting files...')
        with tarfile.open(fname) as tar:
            tar.extractall()
        os.remove(fname)
        return process_cifar()
    
def cifar_for_library(channel_first=True, one_hot=False):
    # Raw data
    x_train, x_test, y_train, y_test = maybe_download_cifar()
    # Scale pixel intensity
    x_train = x_train / 255.0
    x_test = x_test / 255.0
    # Reshape
    x_train = x_train.reshape(-1, 3, 32, 32)
    x_test = x_test.reshape(-1, 3, 32, 32)
    # Channel last
    if not channel_first:
        x_train = np.swapaxes(x_train, 1, 3)
        x_test = np.swapaxes(x_test, 1, 3)
    # One-hot encode y
    if one_hot:
        y_train = np.expand_dims(y_train, axis=-1)
        y_test = np.expand_dims(y_test, axis=-1)
        enc = OneHotEncoder(categorical_features='all')
        fit = enc.fit(y_train)
        y_train = fit.transform(y_train).toarray()
        y_test = fit.transform(y_test).toarray()
    # dtypes
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    return x_train, x_test, y_train, y_test

# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

### Init model

In [ ]:
# Hyperparams
EPOCHS  =  10
BATCHSIZE  =  64
LR  =  0.01
MOMENTUM  =  0.9
N_CLASSES  =  10
GPU  =  True
BATCH_SIZE  =  32

def create_model(n_classes=N_CLASSES):
    model = Sequential()
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu',
                     input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

def init_model(m, lr=LR, momentum=MOMENTUM):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(lr, momentum),
        metrics = ['accuracy'])
    return m

model = create_model()
model = init_model(model)
model.summary()

### Main training loop

In [ ]:
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

### Main evaluation loop

In [ ]:
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))